In [181]:
import nltk
import re
import textdistance
from collections import Counter
import pandas as pd

In [182]:
w = []
with open('cwords.txt', 'r', encoding='utf8') as f:
    file_name_data = f.read()
    file_name_data = file_name_data.lower()
    w = re.findall('\w+', file_name_data)

main_set = set(w)

In [183]:
print(len(main_set))

4974


In [184]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()

def lemmword(word):
    return lemma.lemmatize(word)

In [185]:
# arg2 = 'kings'
# temp2 = lemmword(arg2)
# print(temp2)
# print(type(temp2))

In [186]:
def counting_words(words):
    wordcount = {}
    wordcount = Counter(words)
    for word in words:
        temp = lemmword(word)
        main_set.add(temp)
        if temp in wordcount:
            wordcount[temp] += 1
        else:
            wordcount[temp] = 1;
            
    return wordcount

In [187]:
def cal_prob(wordcount):
    probs = {}
    m = sum(wordcount.values())
    for key in wordcount.keys():
        probs[key] = wordcount[key]/m
    return probs

In [188]:
#delete
def deleteletter(word):
    deletelist = []
    splitlist = []
    
    for i in range(len(word)):
        splitlist.append((word[0:i], word[i:]))
        
    for a, b in splitlist:
        deletelist.append(a + b[1:])
    return deletelist

In [189]:
#swap 2 letters of the word
def switchletter(word):
    splitlist = []
    switchlist = []
    
    for i in range(len(word)):
        splitlist.append((word[0:i], word[i:]))
        
    swithlist = [a + b[1] + b[0] + b[2:] for a, b in splitlist if len(b) >= 2]
    return switchlist

In [190]:
#replace- change one letter to another
def replaceletter(word):
    splitlist = []
    replacelist = []
    
    for i in range(len(word)):
        splitlist.append((word[0:i], word[i:]))
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    replacelist = [a + l + (b[1:] if len(b) > 1 else '') for a, b in splitlist if b for l in alphs]
    return replacelist

In [191]:
#insert- add additional chars
def insertletter(word):
    splitlist = []
    insertlist = []
    
    for i in range(len(word)):
        splitlist.append((word[0:i], word[i:]))
        
    alphs = 'abcdefghijklmnopqrstuvwxyz'
    insertlist = [a + l + b for a, b in splitlist for l in alphs]
    return insertlist

In [192]:
def colab1(word, allow_switches=True):
    colab1 = set()
    colab1.update(deleteletter(word))
    if allow_switches:
        colab1.update(switchletter(word))
    colab1.update(replaceletter(word))
    colab1.update(insertletter(word))
    return colab1


In [193]:
def colab2(word, allow_switches=True):
    colab2 = set()
    edit_one = colab1(word, allow_switches=allow_switches)
    for w in edit_one:
        if w:
            edit_two = colab1(w, allow_switches=allow_switches)
            colab2.update(edit_two)
    return colab2

In [194]:
def getcorrect(word, probs, vocab, n = 2):
    suggestedword = []
    bestsuggestion = []
    suggestedword = list(
        (word in vocab and word)
        or colab1(word).intersection(vocab)
        or colab2(word).intersection(vocab)
    )
    bestsuggestion = [[s, probs[s]] for s in list(reversed(suggestedword))]
    return bestsuggestion

In [195]:
wordcount = counting_words(w)
probs = cal_prob(wordcount)
print(wordcount.most_common()[0:10])

[('the', 4924), ('and', 2620), ('of', 2560), ('i', 2534), ('a', 2035), ('to', 1530), ('in', 1208), ('it', 968), ('that', 914), ('my', 882)]


In [196]:
myword = input("enter: ")
tempcorrections = getcorrect(myword, probs, main_set, 2)

for i, wordprob in enumerate(tempcorrections):
    if(i < 3):
        print(wordprob[0])
    else:
        break;

enter: mokey
money
monkey


In [234]:
def sol(myword):
    
    if wordcount[myword] > 0:
        return 'Your words is correct :)\n'
    
    result = []
    tempcorrections = getcorrect(myword, probs, main_set, 2)

    for i, wordprob in enumerate(tempcorrections):
        if(i < 3):
            result.append(wordprob[0])
#             print(wordprob[0])
        else:
            break;
#     print(result)
        
    sim = [1-(textdistance.Jaccard(qval=2).distance(v,myword)) for v in wordcount.keys()]
    df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
    df = df.rename(columns={'index':'Word', 0:'Prob'})
    df['Similarity'] = sim
    output = df.sort_values(['Similarity', 'Prob'], ascending=False)['Word'].head()
    
    for x in output:
        if x not in result:
            result.append(x)
#     result = set(result)
    return result

In [240]:
test = input('Enter word: ')
print(sol(test))

Enter word: mokey
['money', 'monkey', 'smoke', 'smoked', 'key', 'woke']


In [236]:
import gradio as gr
# appinputs = gr.inputs.Textbox(lines=2, placeholder='enter sentence....')

In [237]:
ifc = gr.Interface(fn=sol, inputs='text', outputs='text', title='Autocorrector')

In [238]:
ifc.launch()

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.
